# Test-driven development (TDD)

Test-driven development (TDD) is a programming paradigm that challenges developers to create tests for their code _before writing the actual code_. This seems like we're putting the cart before the horse!

However, it provides a blueprint for where your code is headed and provides discipline for the applications. We will cover here a number of concepts that are used for TDD in Python. In particular, we have:

  * The `unittest` library
  * The `pytest` library
  * Data synthesis with `mimesis` (covered in class)
  
We will also briefly discuss the merits of Chaos Engineering.

In [30]:
import sys

import unittest
import pytest

In [32]:
help(unittest)

Help on package unittest:

NAME
    unittest

MODULE REFERENCE
    https://docs.python.org/3.6/library/unittest
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    Python unit testing framework, based on Erich Gamma's JUnit and Kent Beck's
    Smalltalk testing framework (used with permission).
    
    This module contains the core framework classes that form the basis of
    specific test cases and suites (TestCase, TestSuite etc.), and also a
    text-based utility class for running the tests and reporting the results
     (TextTestRunner).
    
    Simple usage:
    
        import unittest
    
        class IntegerArithmeticTestCase(unittest.TestCase):
            def testAdd(self):  # test meth

In [33]:
# Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Get Version information
print(sys.version)
print(f"pytest version: {pytest.__version__}")

3.6.7 | packaged by conda-forge | (default, Feb 26 2019, 03:50:56) 
[GCC 7.3.0]
pytest version: 4.3.0


### Unit tests with `unittest`

Inspired by the `JUnit` project (it used to be called `PyUnit`!) the `unittest` library was designed to be similar to testing frameworks found in other languages. It is built for automating tests, analysis of test results and behavior and sharing test meta-data, such as setup information.

A typical test involves:
 *  **Design a class** from `unittest.TestCase`
 *  **Create methods** with a`test_` prefix
 *  **Run tests** with a `unittest.main()` near the end of your code 

The `unittest` library provides the following four concepts: 

 *  **test fixture** --- preparation/construction required to conduct tests, such as establishing database connections, folder structures or servers
 *  **test case** --- the atomic unit under test. A focused analysis on test behavior, using `TestCase`
 *  **test suite** ---  the entire collective of tests to be performed in a single context 
 *  **test runner** --- the interface between the user and the test suite, controlling test execution

(see the `unittest` documentation at https://docs.python.org/3/library/unittest.html)

In [21]:
import unittest

class TestNumerical(unittest.TestCase):
    """Tests a few numerical properties, including positivity,
    whether a number is a float and some addition properties.
    
    This is meant to be run as a program from the command-line.
    Either run with:
        python test_numerical.py -v
    Or, if you are running from a *nix operating system, the above shebang may
    work, so you can instead run:
        ./test_numerical.py -v
    """
    
    def test_positive(self):
        print("test_positive")
        self.assertTrue(3*2 > 0)
        self.assertFalse(-3*2 > 0)
        
    def test_is_float(self):
        print("test_is_float")
        self.assertTrue(type(float(5)) is float)
        self.assertFalse(type(int(5)) is float)
        
    def test_addition(self):
        print("test_addition")
        a = 3
        b = 4
        self.assertEqual(a + b, 7)
        self.assertNotEqual(a + b, 0)
        
        
if __name__ == '__main__':
    unittest.main()

E
ERROR: /run/user/1000/jupyter/kernel-1c7e65a2-da7f-41a9-91dc-7edb45bc43fd (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/run/user/1000/jupyter/kernel-1c7e65a2-da7f-41a9-91dc-7edb45bc43fd'

----------------------------------------------------------------------
Ran 1 test in 0.005s

FAILED (errors=1)


SystemExit: True

/home/ra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Rather than running interactively, this is designed to be automatically run at an appropriate point (such as a new code commit). So we can run the above code as a program thus:

In [ ]:
! python ./code/test_numerical.py -v

Alternatively, you may call the BASH shell, if it is defined on your system:

In [ ]:
%%bash 
./code/test_customer_data.py -v

Note the output of the test. Part of the art is to choose judicious tests that reflect the behavior of the code you wish to test.

You may have noted the various assertions in the test code. The full range offered by `TestCase` are as follows: 

| Method | Checks that |
| --- | --- |
| assertEqual(a, b) | a == b |
| assertNotEqual(a, b) | a != b |
| assertTrue(x) | bool(x) is True |
| assertFalse(x) | bool(x) is False |
| assertIs(a, b) | a is b |
| assertIsNot(a, b) | a is not b |
| assertIsNone(x) | x is None |
| assertIsNotNone(x) | x is not None |
| assertIn(a, b) | a in b |
| assertNotIn(a, b) | a not in b |
| assertIsInstance(a, b) | isinstance(a, b) |
| assertNotIsInstance(a, b) | not isinstance(a, b) |

Does anyone recall/know the difference between equality (`==`) and `is`?

In [22]:
x = [1, 2, 3] 
x == [1, 2, 3]
x is [1, 2, 3] 

x = 3.14159
x == 3.14159
x is 3.14159

# Final---tricky---example
x = 20
x == 20
x is 20

True

### The `pytest` library

We implemented unit tests with `unittest` above. Did it seem like we repeated ourselves a lot in order to perform what at are essentially simple tests? In other words, `unittest` requires a lot of _boilerplate_ code to run. What if we removed most of this boilerplate? Wouldn't that be more fun? This is the motivation behind the `pytest` library.

Assertions are already built into the standard library. This, and other pre-existing code, is leveraged to provide smooth and compact syntax. 

In [23]:
# import unittest
# NOTE: there is no need to import unittest or other special library
# to write the code for pytest

def test_positive():
    print("test_positive")
    assert 3*2 > 0
    assert not -3*2 > 0
    
def test_is_float():
    print("test_is_float")
    assert type(float(5)) is float
    assert not type(int(5)) is float
    
def test_addition():
    print("test_addition")
    a = 3
    b = 4
    assert a + b ==  7
    assert not a + b == 0 

Note how much cleaner the syntax is. There is no need to import the `unittest` (or other) library. No need for a special class, such as `unittest.TestCase` as required above. And no clumsy-looking line near the end of the file to check to see if we are currently running in main().

Let's see how it looks in test:

In [26]:
! python -m pytest -v ./code/test_numerical_pytest.py

============================= test session starts ==============================
platform linux -- Python 3.6.7, pytest-4.3.0, py-1.8.0, pluggy-0.9.0 -- /home/ra/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/ra/host/BH_Analytics/Discover/DataEngineering/notebooks, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.1.3, arraydiff-0.3
collected 3 items                                                              

code/test_numerical_pytest.py::test_positive PASSED                      [ 33%]
code/test_numerical_pytest.py::test_is_float PASSED                      [ 66%]
code/test_numerical_pytest.py::test_addition PASSED                      [100%]

=========================== 3 passed in 0.11 seconds ===========================


In [ ]:
from customer_data import Customer

customer1 = Customer("beeble", "bsmith@itsme.com", 350, 7)
customer2 = Customer("KarenRulz", "kaz@yolo.com", 5, 2)


def test_account_positive():
    print("test_account_positive")
    assert customer1.account_balance >= 0
    assert customer2.account_balance >= 0
    
    
def test_purchase_level():
    print("test_purchase_level")
    customer1.purchase_level(2)
    customer2.purchase_level(1)
    assert customer1.level == 9
    assert customer1.account_balance == 350 - 2*10
    assert customer2.level == 3
    assert customer2.account_balance == 5 - 1*10 

In [27]:
! python -m pytest -v ./code/test_customer_data_pytest.py

============================= test session starts ==============================
platform linux -- Python 3.6.7, pytest-4.3.0, py-1.8.0, pluggy-0.9.0 -- /home/ra/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /home/ra/host/BH_Analytics/Discover/DataEngineering/notebooks, inifile:
plugins: remotedata-0.3.1, openfiles-0.3.1, doctestplus-0.1.3, arraydiff-0.3
collected 2 items                                                              

code/test_customer_data_pytest.py::test_account_positive PASSED          [ 50%]
code/test_customer_data_pytest.py::test_purchase_level PASSED            [100%]

=========================== 2 passed in 0.17 seconds ===========================


### Data synthesis with `mimesis`

An important element of TDD is having a data-set to test. However, some of this will have to occur before you actually have any real data! This is where data synthesis is useful. 

In class, we will make use of the `mimesis` library applied to synthesising the data of customers of a fictitious online game.  

### Chaos engineering

"Chaos Engineering is the discipline of experimenting on a system in order to build confidence in the system’s capability to withstand turbulent conditions in production."
from: https://principlesofchaos.org/

See the O'Reilly book ["Chaos Engineering: Building Confidence in System Behavior through Experiments"](http://www.oreilly.com/webops-perf/free/chaos-engineering.csp) by Casey Rosenthal, Lorin Hochstein, Aaron Blohowiak, Nora Jones and Ali Basiri (original Chaos Engineers at Netflix). 

  * Gremlin
  * [Awesome Chaos Engineering list on GitHub](https://github.com/dastergon/awesome-chaos-engineering)
  * Chaos Toolkit

## Conclusion

We covered a number of key concepts behind Test-Driven Development (TDD). Notably, this was the `unittest` library to perform unit tests of your code. Then we examined the improved syntax and behavior of the `pytest` library. We also looked at synthesizing data with `mimesis`. We finally discussed the concept of Chaos Engineering, with an aim to potential applications within your own work-flow.